### GROUP 21


## ADM_HOMEWORK 3

####   *  ATEFE MORADAN
####   *  GIUSEPPE STEFANALLI
####   *  VIKRANTH ALE


######  Importing required modules to build [*airbnb*](https://www.airbnb.com/) Search Engine 

In [3]:
import pandas as pd
import numpy as np
import csv
import json
import string
import unicodedata
import re
import folium
import geopy
import geopy.distance
from collections import defaultdict
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
from nltk.stem.snowball import SnowballStemmer
from scipy.spatial.distance import cosine
from geopy.geocoders import Nominatim
from geopy.distance import great_circle

### Step 1 :
######  Loading data into a DataFrame for Analysis

In [3]:
df=pd.read_csv("airbnb-property-data-from-texas/Airbnb_Texas_Rentals.csv")
df=df.dropna(axis=0, subset=['description','title'])
df.to_csv("airbnb-property-data-from-texas/Airbnb_Texas_Rentalsd.csv")

### Step 2 :
###### Reading the CSV file and writing the data into TSV files
######        Creating  .tsv file for each record of the dataset into a new Document for corresponding Doc_Id's

In [6]:
with open("airbnb-property-data-from-texas/Airbnb_Texas_Rentalsd.csv","r",encoding="utf8") as csvfile:
    t=csv.reader(csvfile)
    with open("airbnb-property-data-from-texas1/Airbnb_Texas_Rentals1.tsv","w",encoding="utf8") as tsvfile:
        t1=csv.writer(tsvfile,delimiter="\t")
        for line in t:
            t1.writerow(line)

In [7]:
with open("airbnb-property-data-from-texas1/Airbnb_Texas_Rentals1.tsv","r",encoding="utf8") as csvfile:
    t=csv.reader(csvfile)
    i=0
    for line in t:
        if i!=0:
            line=' '.join(line)
            line=line.split('\t')
            with open("airbnb-property-data-from-texas4/doc_"+str(i)+".tsv","w",encoding="utf8") as tsvfile:
                csv.writer(tsvfile,delimiter="\t").writerow(line[1:])
        i+=1 

### Step 3 :
#####  Defining Functions to clean the data for Removing Punctuation, Accents and to Create Stem Sentences inside the DataFrame

In [21]:
def stem_sentences(sentence):
    tokens = sentence.split()
    stemmed_tokens = [stemmer.stem(token) for token in tokens]
    return ' '.join(stemmed_tokens)

In [22]:
o=string.punctuation+'“–'
def remove_punctuations(text):
    for punctuation in o:
        text = text.replace(punctuation, '')
    return text

In [23]:
def remove_accents(input_str):
    nfkd_form = unicodedata.normalize('NFKD', input_str)
    return u"".join([c for c in nfkd_form if not unicodedata.combining(c)])

In [180]:
stemmer = SnowballStemmer("english")
df=pd.DataFrame(columns=['title','description'])
ps = PorterStemmer()
stop = stopwords.words('english')
l = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description' , 'latitude', 'longitude','title','url']
for i in range(1,18255):
    d=pd.read_csv("airbnb-property-data-from-texas4/doc_"+str(i)+".tsv",names=l,delimiter="\t")
    d['title']=d['title'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    d['description']=d['description'].apply(lambda x: ' '.join([word for word in x.split() if word not in (stop)]))
    d['title'] = d['title'].apply(lambda x:''.join([i for i in x 
                                                  if i not in string.punctuation]))
    d['description'] = d['description'].apply(lambda x:''.join([i for i in x 
                                                  if i not in string.punctuation]))
    d['title'] = d['title'].apply(stem_sentences)
    d['description'] = d['description'].apply(stem_sentences)
    d['title'] = d['title'].str.replace('\d+', '')
    d['description'] = d['description'].str.replace('\d+','')
    d['title'] = d['title'].apply(remove_punctuations)
    d['description'] = d['description'].apply(remove_punctuations)
    d['title'] = d['title'].apply(remove_accents)
    d['description'] = d['description'].apply(remove_accents)
    df=df.append(d[['title','description']])
    d.to_csv("airbnb-property-data-from-texas5/doc_"+str(i)+".tsv",header=False,index=False,sep="\t")
    d[['title','description']].to_csv("airbnb-property-data-from-texas6/docI_"+str(i)+".tsv",header=False,index=False,sep="\t")

In [181]:
# Saving the Data into a CSV file after Cleaning it
#This is done to minimise time and efforts in carrying out above repeatative steps 2 - 4

df.to_csv("new_backup_title_desc.csv")

### Step 3.1 :
######  Performing Text Analysis for the Columns ' Title ' & ' Description '  

In [5]:
df1=pd.read_csv("new_backup_title_desc.csv")
#As we decided to merge these two columns and if one column is NAN , if we add them together we will get the NAN
#We decided to fill the na values
df1=df1.fillna("TEXAS")

In [6]:
#merge the title and description columns 
d1 = df1['title']+' '+df1['description']
d2 = pd.DataFrame(d1)
d2.columns = ["words"]

In [7]:
#find the frequency of each word , the following script produce a Series , the index of the Series is the unique
#words in the dataframe
w=d2['words'].str.split(' ', expand=True).stack().value_counts()
wordlist=list(w.index)

In [9]:
# Creating a Dictionary for all the words
#intializing a unique id to each word
termiddic={}
count=0
for i in wordlist:
    termiddic[i]=count
    count+=1
del termiddic['']

In [10]:
# Writing the Dictionary to a Json File
with open('termiddic.json', 'w') as fp:
    json.dump(termiddic, fp)

In [11]:
# Loading the Json File
with open('termiddic.json', 'r') as fp:
    termiddic = json.load(fp)

In [13]:
# create a list of documnets in order to create an inverted index
l=[]
count=0
for i in range(1,18255):
    l1=['title','description']
    f=pd.read_csv("airbnb-property-data-from-texas6/docI_"+str(i)+".tsv",delimiter="\t",names=l1)
    f=f.fillna('TEXAS')
    f1 = f['title']+' '+f['description']
    f2 = pd.DataFrame(f1)
    f2.columns = ["words"]
    w=f2['words'].str.split(' ', expand=True).stack().value_counts()
    a=list(w.index)
    l1=set(a)
    l.append(l1)

#### Step 3.1.1 :
######  Creating an Inverted Index to build a Search Engine

In [14]:
inv_indx = defaultdict(list)
for idx, text in enumerate(l):
    for word in text:
        if word in list(termiddic.keys()):
            value=termiddic[word]
            inv_indx[value].append(idx+1)

In [15]:
inv_indx=dict(inv_indx)

In [16]:
with open('inv_indx.json', 'w') as fp:
    json.dump(inv_indx, fp)

In [17]:
with open('inv_indx.json', 'r') as fp:
    inv_indx = json.load(fp)
inv_indx={int(key):inv_indx[key] for key in inv_indx}

#### Step 3.1.2 :
######  Querying the Search Engine

In [18]:
q1=input()

bedroom house near beach


In [19]:
# Using built-in Library from NLTK for Stop Words
stemmer = SnowballStemmer("english")
df=pd.DataFrame(columns=['title','description'])
ps = PorterStemmer()
stop = stopwords.words('english')

In [24]:
# Filtering the Stopwords,Punctuations,Accents, Stem words from the Search Query

stopword=lambda l1: ' '.join([word for word in l1.split() if word not in (stop)])
punctuation= lambda x:''.join([i for i in x if i not in string.punctuation])
q1=stopword(q1)
q1=punctuation(q1)
q1=stem_sentences(q1)
q1=remove_punctuations(q1)
q1=remove_accents(q1)

In [25]:
# Fetching the TermId's from the dictionary for the terms in Search Query

tid1=[]
for l in (q1.split()):
    if l in termiddic:
        value=termiddic[l]
        tid1.append(value)

In [26]:
# Fetching all the documentID's that contains the Terms in the Search Query from the Inverted Index

docid1=[]
for t in tid1:
    value=inv_indx[t]
    docid1.append(set(value))
u = set.intersection(*docid1)

##### Search Results for the User Query

In [28]:
# Populating the Search Results into a DataFrame with Title , Description , City & Url 

p=pd.DataFrame()
l = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description' , 'latitude', 'longitude','title','url']
for b in u:
    d=pd.read_csv("airbnb-property-data-from-texas4/doc_"+str(b-1)+".tsv",names=l,delimiter="\t")
    p=p.append(d[['description','title','city','url']])

In [29]:
p

,description,title,city,url
14667,This charming one bedroom/one bath guest cotta...,M Streets Cottage just off trendy Greenville Ave,Dallas,https://www.airbnb.com/rooms/13248507?location...
16685,Breathtaking views of Copano Bay from the back...,Forever Paradise,Taft,https://www.airbnb.com/rooms/18699945?location...
2385,Dome in the Dunes can sleep up to 10-or 14 if ...,Ocean View Sleep 9 POOL ask for 10% off M-TH,Port Aransas,https://www.airbnb.com/rooms/6410495?location=...
17720,A retro modern home fit for the Jetsons! Priv...,Jetsons MODsquad +LuxShower!,Austin,https://www.airbnb.com/rooms/14631218?location...
2268,My place is close to Shopping movies water p...,Quaint quiet and comfortable-bedroom,Pflugerville,https://www.airbnb.com/rooms/13842357?location...


### Step 3.2 :
#### Conjunctive query & Ranking score

In [39]:
termdf=pd.DataFrame(list(termiddic.items()), columns=['word','termid'])

In [40]:
termdf=termdf.dropna(axis=0, subset=['word','termid'])

In [41]:
# Calculating the 'Idf'

"""The formula for calculating the idf is the logarithm of all the documents divided by the documents that contain
that word"""

#for each word we calculate the idf  and we assign the idf in the 'idf' column

for i,word in enumerate(termdf['word']):
    if len(d2[d2['words'].str.contains(word)])==0:
           continue
    else:
        
        """ len(d2[d2['words'].str.contains(word) this script calculate the number of documents that contain 
        the desired  words"""
        
           termdf.loc[i, 'idf'] = np.log(d2.shape[0]/(len(d2[d2['words'].str.contains(word)])))
            

In [42]:
x=termdf

In [43]:
#in the dataframe we assign the word column as a index for the following process
x.set_index('word',inplace=True)

#### Step 3.2.1 :
##### Creating a New Inverted Index

In [44]:
"""In the following script we calculated the inverted index in which for each term id we have the tfidf of that word 
                      in each specific document"""
new_invindx={}
for k in termiddic:
    t=termiddic[k]
    l=inv_indx[t]
    new_invindx[t]=[]
    for i in l:
        if i in d2.index:
            d3=d2.loc[(i-1),'words']
            t1=d3.split(" ")
            
            #here we find the frequency of the word in the documnet
            
            c=t1.count(k)
            tf=c/len(t1)
            tfidf=(x.loc[k]['idf'])*tf
            new_invindx[t].append((i,tfidf))
        else:
            continue
            

In [45]:
with open('new_invindx.json', 'w') as fp:
    json.dump(new_invindx, fp)

In [46]:
with open('new_invindx.json', 'r') as fp:
    new_invindx1= json.load(fp)
new_invindx1={int(key):new_invindx1[key] for key in new_invindx1}

In [47]:
new_invindx=new_invindx1

#### Step 3.2.2 :
######  Querying the Search Engine

In [81]:
q=input()

house garden beach bedroom


In [82]:
# Using built-in Library from NLTK for Stop Words

stemmer = SnowballStemmer("english")
df=pd.DataFrame(columns=['title','description'])
ps = PorterStemmer()
stop = stopwords.words('english')

In [83]:
# Filtering the Stopwords,Punctuations,Accents, Stem words from the Search Query

stopword=lambda l1: ' '.join([word for word in l1.split() if word not in (stop)])
punctuation= lambda x:''.join([i for i in x if i not in string.punctuation])
q=stopword(q)
q=punctuation(q)
q=stem_sentences(q)
q=remove_punctuations(q)
q=remove_accents(q)

In [84]:
# Fetching the TermId's from the dictionary for the terms in Search Query

tid=[]
for l in (q.split()):
    if l in termiddic:
        value=termiddic[l]
        tid.append(value)

In [115]:
# Fetching all the documentID's that contains the Terms in the Search Query from the Inverted Index

docid=[]
for t in tid:
    value=inv_indx[t]
    docid.append(set(value))
u = set.intersection(*docid)

In [86]:
# Search Results with Document ID's 

u

{1673, 13598}

In [54]:
Q=q.split(" ")

In [55]:
# Document ID's with search Terms are Saved in Dictinary based on their TF-IDF score

d1={}
for h in Q:
    c1=Q.count(h)
    tf=c1/len(Q)
    if h in termiddic:
        d1[termiddic[h]]=(x.loc[h]['idf'])*tf
    else:
        continue

In [56]:
d1

{6: 0.29366937231741014,
 8: 0.27439299056008454,
 53: 0.61977912467258534,
 229: 0.84926063847630684}

In [57]:
# Calculating the Cosine Similarity Ranked by their Score

""" In the following script we calculate the cosine similarity between query and documents that contain all the 
                            words of the query"""
precosl={}
finalcosl={}
for d in list(u):
    temp=d1
    g=d2.loc[(d-1),'words']
    g=g.split(" ")
    for i in g:
        if i in termiddic:
            k=termiddic[i]
            tl=new_invindx[k]
            for r in tl:
                if r[0]==d:
                    precosl[termiddic[i]]=r[1]

""" Based on the formula of cosine similarity in the following script we  should have two vectors of the same size
    therefore we assign a key(here is termid) in the dictionary with the value 0 if the term is not in the query
    but it is in the document"""

    for m in precosl:
        if m in temp:
            continue
        else:
            temp[m]=0
    doc_d= pd.Series(precosl)
    query_d = pd.Series(temp)
    finalcosl[d]=(1 - cosine(doc_d, query_d))

In [105]:
finalcosl

{1673: 0.44025203786575107, 13598: 0.17349905600673843}

##### Search Results of document ID's ranked with their Cosine Similarity 

In [116]:
p1=pd.DataFrame()
l = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description' , 'latitude', 'longitude','title','url']
for b in u:
    d=pd.read_csv("airbnb-property-data-from-texas5/doc_"+str(b)+".tsv",names=l,delimiter="\t")
    d['Cosine-Similiarty']=finalcosl[b]
    p1=p1.append(d[['description','title','city','url','Cosine-Similiarty']])

In [117]:
p1.sort_values(by=['Cosine-Similiarty'],ascending=False, kind='heapsort')

,description,title,city,url,Cosine-Similiarty
0,new hous one block beach this bedroom bath b...,vista breez,Galveston,https://www.airbnb.com/rooms/18371258?location...,0.440252
0,this one kind sqft townhous bedroom locat hi...,stun california inspir modern beach townhom,Dallas,https://www.airbnb.com/rooms/15510582?location...,0.173499


### Step 4 :
##### Defining a new score !

In [118]:
df1=pd.read_csv("airbnb-property-data-from-texas/Airbnb_Texas_Rentalsd.csv")

In [119]:
l = ['average_rate_per_night', 'bedrooms_count', 'city', 'date_of_listing', 'description' , 'latitude', 'longitude','title','url']
Df=pd.DataFrame(columns=l)
for i in range(1,18255):
    ds=pd.read_csv("airbnb-property-data-from-texas5/doc_"+str(i)+".tsv",delimiter="\t",names=l)
    Df=Df.append(ds)

In [120]:
Df=Df.reset_index(drop=True)
Df=Df.dropna(axis=0, subset=['bedrooms_count','average_rate_per_night'])

In [121]:
#Cleaning the data and extracting meaningful information to define New Score based on City,No of Bedrooms & Average rate/Night
#we remove the $ sign in average_rate_per_night and convert it to float

Df['average_rate_per_night']=Df['average_rate_per_night'].apply(lambda x : int(x.replace('$','')))
Df['bedrooms_count']=Df['bedrooms_count'].apply(lambda x : str(x))

#we convert the bedroom to float and then we replace the Studio with 1

Df['bedrooms_count']=Df['bedrooms_count'].apply(lambda x : float(x.replace('Studio','1')))

######  Querying the Search Engine with City , Number of BedRooms & Average rate / Night 

In [122]:
c=input('please Enter city: ')

please Enter city: Galveston


In [123]:
b=input('please Enter the number of bedrooms? ')

please Enter the number of bedrooms? 2


In [124]:
if b !="Studio":
    b=int(b)
if b=="Studio":
    b=1

In [125]:
r=input('please Enter the number of average_rate_per_night (in dollar)? ')
r=int(r.replace('$',''))

please Enter the number of average_rate_per_night (in dollar)? 50


In [126]:
u

{1673, 13598}

In [127]:
""" u is the intersection of documents that contains the word from previous part in other words it contains all 
the documents that contains all the words in the query"""

u = [x-1 for x in u]
S=Df.ix[u]


In [128]:
S=S.reset_index(drop=True)

In [129]:
""" Using Builtin library Geopy & Nominatim in order to find the distance between the city that is in the query
and the city in the documents """

geolocator = Nominatim(user_agent='myapplication')
try:
    location = geolocator.geocode(c,timeout=None)
    lan=location.latitude
    lon=location.longitude
except GeocoderTimedOut as e:
    lan=100
Y=(lan,lon)

In [92]:
"""Is the Longtitude and latitude of the city in the query"""
Y

(29.7355047, -94.9774274)

In [130]:
""" The w vector contains the score , 0.1 is for bedroom , 0.3 is for the price and 0.6 for the city and 0.4 for
   the distance of the city and other cities """

""" The document will get the score of bedroom if the bedroom is one point less or more than the bedroom entered """

""" The document will get the score of price if the price is 20 dollar  less or more than the price that
entered by the user """

""" The document will get the score of city if the city is excatly as the user entered """

""" If there isn't any city the same as the user entered we calculate the distance and if we find a house that is 
20 km away then the location that the user entered in this document will give the score of the distance """

w=[0.1,0.3,0.6,0.4]
for i in range(len(S)):
    m=[]
    
    if b-1<=S.loc[i,'bedrooms_count']<=b+1:
        m.append(1)
    else:
        m.append(0)
    if r-20<=S.loc[i,'average_rate_per_night']<=r+10:
        m.append(1)
    else:
        m.append(0)
    if c==S.loc[i,'city']:
        m.append(1)
        m.append(0)
    else:
        m.append(0)
        X=(S.loc[i,'latitude'],S.loc[i,'longitude'])
        Dist=great_circle(X,Y).km
        if Dist<=10:
            m.append(1)
        else:
            m.append(0)
    DotProduct = np.dot(m,w)
    S.loc[i,'Score']=DotProduct

#### Search results with a New Score

In [131]:
S.sort_values(by=['Score'],ascending=False, kind='heapsort')

,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,Score
0,200,3.0,Galveston,May 2016,new hous one block beach this bedroom bath b...,29.201620,-94.944485,vista breez,https://www.airbnb.com/rooms/18371258?location...,0.7
1,199,2.0,Dallas,December 2014,this one kind sqft townhous bedroom locat hi...,32.919723,-96.845536,stun california inspir modern beach townhom,https://www.airbnb.com/rooms/15510582?location...,0.1


### Bonus Step : 
##### Making a nice visualization  Listing  all the search results within a given radius !

In [2]:
# Loading Data and creating a new Column with Coordinates information for a particular listing

df=pd.read_csv("Airbnb_Texas_Rentals.csv")

df["coordinates"]= df.apply(lambda x: [x.latitude, x.longitude], axis=1)
                                                                        

In [3]:
# Cleaning the data for missing values and Resetting the Index

df["coordinates"]= df["coordinates"].apply(lambda v: pd.Series(v).dropna().values)  
df["coordinates"]=df["coordinates"][df["coordinates"].apply (len) > 0]
df= df.loc[df.coordinates.notnull()]
df.index = range(len(df.index))


In [4]:
df.head()

,Unnamed: 0,average_rate_per_night,bedrooms_count,city,date_of_listing,description,latitude,longitude,title,url,coordinates
0,1,$27,2,Humble,May 2016,Welcome to stay in private room with queen bed...,30.020138,-95.293996,2 Private rooms/bathroom 10min from IAH airport,https://www.airbnb.com/rooms/18520444?location...,"[30.0201379199512, -95.29399600425128]"
1,2,$149,4,San Antonio,November 2010,"Stylish, fully remodeled home in upscale NW – ...",29.503068,-98.447688,Unique Location! Alamo Heights - Designer Insp...,https://www.airbnb.com/rooms/17481455?location...,"[29.503067675606104, -98.4476879378504]"
2,3,$59,1,Houston,January 2017,'River house on island close to the city' \nA ...,29.829352,-95.081549,River house near the city,https://www.airbnb.com/rooms/16926307?location...,"[29.829352227214898, -95.0815494887563]"
3,4,$60,1,Bryan,February 2016,Private bedroom in a cute little home situated...,30.637304,-96.337846,Private Room Close to Campus,https://www.airbnb.com/rooms/11839729?location...,"[30.6373042787676, -96.33784597296308]"
4,5,$75,2,Fort Worth,February 2017,Welcome to our original 1920's home. We recent...,32.747097,-97.286434,The Porch,https://www.airbnb.com/rooms/17325114?location...,"[32.7470973543511, -97.2864343970125]"


#### Querying the Search Engine to display Properties for a Location

In [5]:
# We have enhanced our search Engine by giving flexibility to use to search by place rather than manual input of coordinates

#Using Libraries to get the coordinates of a by Searching for a particular building, street or a desired place(Manual Text Input)

geolocator= Nominatim()
address=input(" Type Name of building, Street or Place : ")
location= geolocator.geocode(address)
cen_alt=[]
cen_alt.append(location.latitude)
cen_alt.append(location.longitude)

print(cen_alt)
rad=float(input(" Type Radius in metres: "))

C:\Users\VICKY\Anaconda3\lib\site-packages\geopy\geocoders\osm.py:143: UserWarning: Using Nominatim with the default "geopy/1.17.0" `user_agent` is strongly discouraged, as it violates Nominatim's ToS https://operations.osmfoundation.org/policies/nominatim/ and may possibly cause 403 and 429 HTTP errors. Please specify a custom `user_agent` with `Nominatim(user_agent="my-application")` or by overriding the default `user_agent`: `geopy.geocoders.options.default_user_agent = "my-application"`. In geopy 2.0 this will become an exception.
  UserWarning


 Type Name of building, Street or Place : Humble
[29.9988312, -95.2621553]
 Type Radius in metres: 10000


In [6]:
# Creating a Circle within the given radius for search criteria


m = folium.Map(location=cen_alt, zoom_start=12)
circle = folium.Circle(
    radius=rad, # circle radius
    location=cen_alt, # circle center
    color='crimson',
    fill=True, 
).add_to(m) 


            
tooltip = 'Click me!'
mark=folium.Marker(cen_alt, popup= address, tooltip=tooltip, icon=folium.Icon(color="red")).add_to(m)

In [7]:
m

#### Displaying the Houses on Airbnb and directing the User to the landing page of advertisement !

In [8]:
# Displaying Total Houses inside the circle for Search Criteria 
# Initialising the Count and Display with Zero

dis=0
count=0
j=0
tooltip = 'Click me!'
for i in df["coordinates"]:
    dis=(geopy.distance.vincenty(i, cen_alt).km)*1000
    if dis<=rad:
        url=df["url"].loc[j]
        price= df["average_rate_per_night"].loc[j]
        popup = folium.Popup('<a href="' + url + '"target="_blank">' + price + '</a>')
        folium.Marker(i, popup=popup, tooltip=tooltip).add_to(m)
        count+=1
        
    j+=1

In [9]:
m.save("Humble_SearchResults.html")

In [10]:
 # Displaying the Total Count of Houses found within the given Radius

print(count,"Total Houses found for ",address)

m

138 Total Houses found for  Humble
